**[Yüz ifadesi yarışmasına](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge) hoş geldiniz.**

<table>
    <tr>
        <td><img src="https://i.ibb.co/B394D0t/12.png" width="100%" height="100%"></td>
        <td><img src="https://i.ibb.co/5xV8MZj/40.png" width="100%" height="100%"></td>
        <td><img src="https://i.ibb.co/bFgXc43/19.png" width="100%" height="100%"></td>
    </tr>
</table>
Veriler, yüzlerin 48x48 piksel gri tonlamalı görüntülerinden oluşur. Yüzler otomatik olarak kaydedilir, böylece yüz aşağı yukarı ortalanır ve her görüntüde yaklaşık olarak aynı miktarda yer kaplar. 
Görev, her yüzü, yüz ifadesinde gösterilen duyguya göre yedi kategoriden birinde kategorize etmektir:

| categorie  | emotion  |
|:---:|:---|
| 0  | Angry (kızgın)  |
| 1  |  Disgust (nefret) |
| 2  |  Fear (korku) |
| 3  |  Happy (mutlu) |
| 4  |  Sad (üzgün) |
| 5  |  Surprise (şaşırma) |
| 6  | Neutral (doğal) |

Basit bir CNN modeli tanımlıyoruz ve tahmin edilen sonuçları verilen etiketlerle karşılaştırıyoruz.


## Kütüphaneler
Bazı standart kitaplıkları ve sklearn ve keras paketlerini yüklüyoruz.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import tensorflow as tf

#from keras import models
#from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
#from keras.optimizers import RMSprop,Adam
#from keras.utils import to_categorical

## Veri Kümesi Dosya Yolu

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!ls '/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part2/Day11-DeepLearning/notebooks/datasets/emotion_recognition/fer2013'

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part2/Day11-DeepLearning/notebooks/"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART2/Day11-DeepLearning/notebooks/"

DATASET_PATH = ROOT_DIR + "datasets/emotion_recognition/fer2013/"

## Veri Kümesini Yükleme

In [ ]:
data = pd.read_csv(DATASET_PATH+'fer2013.csv')

In [ ]:
data.head()

# Yardımcı Fonksiyonlar
Verileri hazırlamak ve çizmek için bazı yardımcı fonksiyonlar tanımladık.

In [ ]:
#@title Yardımcı Fonksiyonlar

def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

def plot_examples(label=0):
    fig, axs = plt.subplots(1, 5, figsize=(25, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(5):
        idx = data[data['emotion']==label].index[i]
        axs[i].imshow(train_images[idx][:,:,0], cmap='gray')
        axs[i].set_title(emotions[train_labels[idx].argmax()])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        
def plot_all_emotions():
    fig, axs = plt.subplots(1, 7, figsize=(30, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(7):
        idx = data[data['emotion']==i].index[i]
        axs[i].imshow(train_images[idx][:,:,0], cmap='gray')
        axs[i].set_title(emotions[train_labels[idx].argmax()])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        
def plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels, image_number):
    """ Function to plot the image and compare the prediction results with the label """
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    
    bar_label = emotions.values()
    
    axs[0].imshow(test_image_array[image_number], 'gray')
    axs[0].set_title(emotions[test_image_label[image_number]])
    
    axs[1].bar(bar_label, pred_test_labels[image_number], color='orange', alpha=0.7)
    axs[1].grid()
    
    plt.show()
    
def plot_compare_distributions(array1, array2, title1='', title2=''):
    df_array1 = pd.DataFrame()
    df_array2 = pd.DataFrame()
    df_array1['emotion'] = array1.argmax(axis=1)
    df_array2['emotion'] = array2.argmax(axis=1)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    x = emotions.values()
    
    y = df_array1['emotion'].value_counts()
    keys_missed = list(set(emotions.keys()).difference(set(y.keys())))
    for key_missed in keys_missed:
        y[key_missed] = 0
    axs[0].bar(x, y.sort_index(), color='orange')
    axs[0].set_title(title1)
    axs[0].grid()
    
    y = df_array2['emotion'].value_counts()
    keys_missed = list(set(emotions.keys()).difference(set(y.keys())))
    for key_missed in keys_missed:
        y[key_missed] = 0
    axs[1].bar(x, y.sort_index())
    axs[1].set_title(title2)
    axs[1].grid()
    
    plt.show()

## Genel Bakış

In [ ]:
data['Usage'].value_counts()

## Verileri Hazırlama

In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

Eğitim, doğrulama ve test kümelerini tanımlayın:

In [ ]:
train_image_array, train_image_label = prepare_data(data[data['Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data['Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data['Usage']=='PublicTest'])

Görüntüleri yeniden şekillendirin ve ölçeklendirin:

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255

val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255

test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

Hedef değerin kodlaması:

In [ ]:
train_labels = tf.keras.utils.to_categorical(train_image_label)
val_labels = tf.keras.utils.to_categorical(val_image_label)
test_labels = tf.keras.utils.to_categorical(test_image_label)

## Bazı Örnekler

### Tüm Duygular

In [ ]:
plot_all_emotions()

### Kızgın (Angry)

In [ ]:
plot_examples(label=0)

### Nefret (Disgust)

In [ ]:
plot_examples(label=1)

### Korku (Fear)

In [ ]:
plot_examples(label=2)

### Mutlu (Happy)

In [ ]:
plot_examples(label=3)


### Üzgün (Sad)

In [ ]:
plot_examples(label=4)

### Şaşırma (Surprise)

In [ ]:
plot_examples(label=5)

### Doğal(Neutral)

In [ ]:
plot_examples(label=6)

## Etiketlerin Dağıtımı

In [ ]:
plot_compare_distributions(train_labels, val_labels, title1='train labels', title2='val labels')

## Sınıf Ağırlıkları
Etiket dağılımının sınıf ağırlıklarını hesaplayın:

In [ ]:
data[data['Usage']=='Training']['emotion']

In [ ]:
class_weight = dict(zip(range(0, 7), (((data[data['Usage']=='Training']['emotion'].value_counts()).sort_index()) / len(data[data['Usage']=='Training']['emotion'])).tolist()))
class_weight

## Model
Basit bir CNN modeli tanımlıyoruz:

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(tf.keras.layers.MaxPool2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = class_weight,
                    epochs=12,
                    batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test caccuracy:', test_acc)

## Yakınsama Analizi

In [ ]:
loss = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'bo', label='loss_train')
plt.plot(epochs, loss_val, 'b', label='loss_val')
plt.title('value of the loss function')
plt.xlabel('epochs')
plt.ylabel('value of the loss function')
plt.legend()
plt.grid()
plt.show()

In [ ]:
acc = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(1, len(loss)+1)
plt.plot(epochs, acc, 'bo', label='accuracy_train')
plt.plot(epochs, acc_val, 'b', label='accuracy_val')
plt.title('accuracy')
plt.xlabel('epochs')
plt.ylabel('value of accuracy')
plt.legend()
plt.grid()
plt.show()

## Sonuç Analizi

In [ ]:
#Prediction
pred_test_labels = model.predict(test_image_array)

In [ ]:
plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels, 106)

In [ ]:
plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels, 40)

In [ ]:
plot_compare_distributions(test_labels, pred_test_labels, title1='test labels', title2='predict labels')

## Yanlış Tahmini Analiz Edin
Test kümesinde doğruluk puanı yaklaşık %54'tür. 
Bu yüzden yanlış tahminlere odaklanmak açıktır. 
Modeli geliştirmek için ayrıntıları çıkarmak istiyoruz.

In [ ]:
df_compare = pd.DataFrame()
df_compare['real'] = test_labels.argmax(axis=1)
df_compare['pred'] = pred_test_labels.argmax(axis=1)
df_compare['wrong'] = np.where(df_compare['real']!=df_compare['pred'], 1, 0)

In [ ]:
emotions.values()

In [ ]:
conf_mat = confusion_matrix(test_labels.argmax(axis=1), pred_test_labels.argmax(axis=1))

fig, ax = plot_confusion_matrix(conf_mat=conf_mat,
                                show_normed=True,
                                show_absolute=False,
                                figsize=(8, 8))
fig.show()